In [ ]:
import sys
sys.path.append('../util')

from util import get_max_db_id
from util import get_time_span
from util import get_all_url

In [ ]:
target_directory = f"segments_cs"

In [ ]:
STT_CS_GF = 6
STT_CS_GA = 4
STT_CS_GB = 3

group_id = STT_CS_GF
last_db_id = get_max_db_id()

In [ ]:
from transformers import pipeline
from tqdm.auto import tqdm

from pathlib import Path

target_path = Path(target_directory)

rows = []
generator = pipeline(model="openpecha/wav2vec2_run8")

for file in tqdm(target_path.glob('*.wav'), total=len(list(target_path.glob('*.wav')))):
    inf = generator(str(file))["text"]
    rows.append([file.stem, f"https://d38pmlk0v88drf.cloudfront.net/wav16k/{file.name}", inf, get_time_span(str(file.name))])

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [ ]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

In [ ]:
df = df.sort_values('file_name')
df = df.reset_index(drop=True)

In [ ]:
df['group_id'] = group_id
df['state'] = 'transcribing'
df['id'] = df.index + last_db_id + 1

In [ ]:
df.head()

In [ ]:
df['file_name'] = df['file_name'].str.replace(".wav", "")

In [ ]:
urls = get_all_url()

In [ ]:
df = df[~df['url'].isin(urls)]

In [ ]:
# len('STT_CS-D-2017-P-D-B01-1-R-06')

In [ ]:
# df = df[df['file_name'].str[0:28] != 'STT_CS-D-2017-P-D-B01-1-R-06']

In [ ]:
df.to_csv(f"segments_cs_{group_id}.csv", index=False)